In [1]:
#!/usr/bin/env python3
import os
import glob
import json
import shutil
import xml.etree.ElementTree as ET

In [2]:
# === User-configurable settings ===
CLASS_LIST = ["tortoise", "donkey"]  # Update with your classes
SPLITS = ["train", "val", "test"]
DATA_ROOT = "data"              # Root folder containing train/, val/, test/
ANNOTATIONS_DIR = "annotations" # Folder containing all XML annotation files
COCO_ROOT = os.path.join(DATA_ROOT, "COCO")  # Output COCO directory structure

In [3]:
# === Helper functions ===
def ensure_dirs_exist(path):
    os.makedirs(path, exist_ok=True)

In [4]:
def copy_images(split):
    """Copy images from data/<split>/ to COCO structure."""
    src_dir = os.path.join(DATA_ROOT, split)
    dst_dir = os.path.join(COCO_ROOT, "images", f"{split}2017")
    ensure_dirs_exist(dst_dir)
    for img_file in glob.glob(os.path.join(src_dir, "*.*")):
        shutil.copy(img_file, dst_dir)
    print(f"Copied {len(os.listdir(dst_dir))} images for split '{split}'")
    return dst_dir

In [5]:
def parse_voc_xml(xml_file, class_list):
    """Parse one VOC XML file to extract width, height, and object bboxes."""
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find("size")
    width = int(size.findtext("width"))
    height = int(size.findtext("height"))
    objects = []
    for obj in root.findall("object"):
        cls = obj.findtext("name")
        if cls not in class_list:
            continue
        cat_id = class_list.index(cls) + 1
        bnd = obj.find("bndbox")
        xmin = int(bnd.findtext("xmin"))
        ymin = int(bnd.findtext("ymin"))
        xmax = int(bnd.findtext("xmax"))
        ymax = int(bnd.findtext("ymax"))
        w = xmax - xmin
        h = ymax - ymin
        objects.append({
            "category_id": cat_id,
            "bbox": [xmin, ymin, w, h],
            "area": w * h,
            "iscrowd": 0
        })
    return width, height, objects

In [6]:
def build_coco_data(split):
    """
    Build COCO 'images' and 'annotations' lists by matching images in a split
    with XMLs in the annotations folder.
    """
    images = []
    annotations = []
    ann_id = 1
    img_dir = os.path.join(DATA_ROOT, split)
    xml_dir = os.path.join(DATA_ROOT, ANNOTATIONS_DIR)

    # For each image in the split folder
    for img_id, img_path in enumerate(glob.glob(os.path.join(img_dir, "*.*")), start=1):
        filename = os.path.basename(img_path)
        name, _ = os.path.splitext(filename)
        xml_path = os.path.join(xml_dir, f"{name}.xml")
        if not os.path.exists(xml_path):
            print(f"Warning: Annotation missing for {filename}, skipping. {name} {xml_path}")
            continue
        width, height, objs = parse_voc_xml(xml_path, CLASS_LIST)
        images.append({"id": img_id, "file_name": filename, "width": width, "height": height})
        for obj in objs:
            obj.update({"id": ann_id, "image_id": img_id})
            annotations.append(obj)
            ann_id += 1

    return images, annotations

In [7]:
def write_coco_json(split, images, annotations):
    """Write COCO JSON file for a given split."""
    ann_dir = os.path.join(COCO_ROOT, "annotations")
    ensure_dirs_exist(ann_dir)
    categories = [{"id": idx+1, "name": name, "supercategory": "none"}
                  for idx, name in enumerate(CLASS_LIST)]
    coco = {"images": images, "annotations": annotations, "categories": categories}
    json_path = os.path.join(ann_dir, f"instances_{split}2017.json")
    with open(json_path, "w") as f:
        json.dump(coco, f, indent=2)
    print(f"Wrote COCO JSON: {json_path} with {len(images)} images and {len(annotations)} annotations.")

In [8]:
def convert_split(split):
    print(f"--- Converting split: {split} ---")
    copy_images(split)
    images, annotations = build_coco_data(split)
    write_coco_json(split, images, annotations)

In [9]:
def main():
    ensure_dirs_exist(os.path.join(COCO_ROOT, "images"))
    for split in SPLITS:
        convert_split(split)
    print("All splits converted.")

In [10]:
main()

--- Converting split: train ---
Copied 349 images for split 'train'
Wrote COCO JSON: data/COCO/annotations/instances_train2017.json with 349 images and 435 annotations.
--- Converting split: val ---
Copied 75 images for split 'val'
Wrote COCO JSON: data/COCO/annotations/instances_val2017.json with 75 images and 94 annotations.
--- Converting split: test ---
Copied 75 images for split 'test'
Wrote COCO JSON: data/COCO/annotations/instances_test2017.json with 75 images and 96 annotations.
All splits converted.
